# Kubernetes Cluster Versions Report

This report loops through all the clusters we maintain, and reports their master and node versions

In [ ]:
from deployer.helm_upgrade_decision import get_all_cluster_yaml_files
from deployer.cluster import Cluster
import subprocess
import json
from tabulate import tabulate

from ruamel.yaml import YAML
yaml = YAML(typ='safe')

In [ ]:
cluster_files = get_all_cluster_yaml_files()

clusters = []
for cf in cluster_files:
    with open(cf) as f:
        cluster = Cluster(yaml.load(f), cf.parent)
    clusters.append(cluster)

In [ ]:
versions = {}
for c in clusters:
    with c.auth():
        cluster_name = c.spec['name']
        # Populate cloud provider type
        versions[cluster_name] = {'provider': c.spec['provider']}
        
        # Get master version
        version_output = json.loads(subprocess.check_output(['kubectl', 'version', '--output', 'json']))
        server_version = version_output["serverVersion"]["gitVersion"]
        versions[cluster_name]['master'] = server_version.split('-')[0]
        
        # Get node versions
        node_output = json.loads(subprocess.check_output(["kubectl", "get", "node", "--output", "json"]))
        node_versions = set([n["status"]["nodeInfo"]["kubeletVersion"].split('-')[0] for n in node_output["items"]])
        versions[cluster_name]['nodes'] = node_versions
        
        

In [ ]:
headers = ('provider', 'cluster', 'master version', 'node version(s)')
tabulate([
    (version['provider'], name, version['master'], ','.join(version['nodes']))
    for name, version in versions.items()
], headers, tablefmt="html")
